# CSV → Planilhas CCIRAS

Gera planilhas de monitoramento para o CCIRAS a partir dos CSVs `vigiram-*.csv` e das contagens de pacientes positivos/negativos. O fluxo utiliza os mesmos arquivos de entrada já exportados do Metabase.

In [1]:
# Célula 1: Configuração de ambiente e diretórios
import pandas as pd
from pathlib import Path

try:
    NB_DIR = Path(__file__).parent.resolve()
except NameError:
    NB_DIR = Path().resolve()  # Jupyter

REPO_ROOT = NB_DIR.parent
DATA_DIR = (REPO_ROOT / "data").resolve()
OUT_DIR = (REPO_ROOT / "outputs").resolve()
DATA_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

print("DATA_DIR =", DATA_DIR)
print("OUT_DIR =", OUT_DIR)

DATA_DIR = C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\data
OUT_DIR = C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs


In [2]:
# Célula 2: Função de leitura robusta de CSV
def read_csv_robusto(path_csv: str, sep: str = ",") -> pd.DataFrame:
    for enc in ("utf-8-sig", "utf-8", "latin-1", "cp1252"):
        try:
            return pd.read_csv(
                path_csv, sep=sep, dtype=str, encoding=enc, engine="python"
            )
        except UnicodeDecodeError:
            continue
    return pd.read_csv(
        path_csv,
        sep=sep,
        dtype=str,
        encoding="utf-8",
        encoding_errors="replace",
        engine="python",
    )

In [3]:
# Célula 3: Planilha de Isolados Detalhados
def planilha_isolados(df: pd.DataFrame) -> pd.DataFrame:
    cols = {
        "data_area_executora": "Data",
        "paciente": "Paciente",
        "solicitacao": "Prontuario",
        "unidade_solicitante": "Unidade/Setor",
        "desc_material_analise": "Tipo de Amostra",
        "microorganismo": "Microrganismo",
        "Antibiótico": "Antimicrobiano Testado",
        "RSI": "Resultado",
        "Perfil de Resistência": "Observações",
    }
    existe = [c for c in cols if c in df.columns]
    return df[existe].rename(columns=cols)

In [4]:
# Célula 4: Planilha de Sensibilidade por Microrganismo
def planilha_sensibilidade(df: pd.DataFrame) -> pd.DataFrame:
    base = df.dropna(subset=["microorganismo", "Antibiótico", "RSI"])
    tab = base.pivot_table(
        index=["microorganismo", "Antibiótico"],
        columns="RSI",
        values="solicitacao",
        aggfunc="count",
        fill_value=0,
    ).reset_index()

    outcome_cols = tab.columns.drop(["microorganismo", "Antibiótico"])
    for col in outcome_cols:
        tab[col] = pd.to_numeric(tab[col], errors="coerce").fillna(0)

    tab["Nº Testes"] = tab[outcome_cols].sum(axis=1)

    for col in ["S", "R", "I"]:
        if col not in tab:
            tab[col] = 0

    tab["% Sensíveis"] = (tab["S"] / tab["Nº Testes"].replace(0, pd.NA) * 100).round(2)
    tab["% Resistentes"] = (tab["R"] / tab["Nº Testes"].replace(0, pd.NA) * 100).round(
        2
    )
    tab["% Intermediários"] = (
        tab["I"] / tab["Nº Testes"].replace(0, pd.NA) * 100
    ).round(2)

    return tab[
        [
            "microorganismo",
            "Antibiótico",
            "Nº Testes",
            "% Sensíveis",
            "% Resistentes",
            "% Intermediários",
        ]
    ]

In [5]:
# Célula 5: Planilha de Tendência Temporal
def planilha_tendencia(df: pd.DataFrame) -> pd.DataFrame:
    if "data_area_executora" not in df.columns:
        raise ValueError("coluna 'data_area_executora' ausente")
    dt = pd.to_datetime(df["data_area_executora"], errors="coerce")
    df = df.assign(Ano=dt.dt.year, Mes=dt.dt.month)
    iso = (
        df.groupby(["Ano", "Mes", "microorganismo"])["solicitacao"]
        .nunique()
        .reset_index(name="Total de Isolados")
    )
    res = (
        df[df["RSI"].str.upper().eq("R")]
        .groupby(["Ano", "Mes", "microorganismo"])
        .size()
        .reset_index(name="Resistentes")
    )
    out = iso.merge(res, on=["Ano", "Mes", "microorganismo"], how="left").fillna(0)
    out["% Resistentes"] = (
        out["Resistentes"] / out["Total de Isolados"].replace(0, pd.NA) * 100
    ).round(2)
    return out

In [ ]:
# Célula 6: Carregamento e Processamento das Contagens (LÓGICA CORRIGIDA)
def carregar_contagens(
    caminho_csv_positivos: Path, caminho_csv_negativos: Path
) -> tuple[int, int]:
    """
    Lê os arquivos de contagem e retorna o número total de pacientes
    positivos e negativos (total de linhas).
    """
    df_pos = read_csv_robusto(caminho_csv_positivos)
    df_neg = read_csv_robusto(caminho_csv_negativos)

    # Retorna o número de registros (linhas) em cada arquivo
    total_positivos = len(df_pos)
    total_negativos = len(df_neg)

    return total_positivos, total_negativos

In [ ]:
# Célula 7: Planilha de Indicadores Epidemiológicos (LÓGICA CORRIGIDA)
def planilha_indicadores(
    df: pd.DataFrame, total_pacientes_positivos: int, total_pacientes_negativos: int
) -> pd.DataFrame:
    """
    Calcula indicadores epidemiológicos agregados para o hospital.
    """
    # Calcula o total de pacientes que tiveram amostras coletadas
    total_pacientes_com_amostra = total_pacientes_positivos + total_pacientes_negativos

    # Agrupa o dataframe principal por microrganismo para contar isolados e resistentes
    iso = (
        df.groupby("microorganismo")["solicitacao"]
        .nunique()
        .reset_index(name="total_isolados")
    )
    res = (
        df[df["RSI"].str.upper().eq("R")]
        .groupby("microorganismo")
        .size()
        .reset_index(name="resistentes")
    )

    # Junta as contagens de isolados e resistentes
    out = iso.merge(res, on="microorganismo", how="left").fillna(0)

    # Calcula as taxas usando o total de pacientes do hospital
    # Adicionado um controle para evitar divisão por zero se não houver pacientes
    if total_pacientes_com_amostra > 0:
        out["Taxa de Incidência (por 1000 pacientes)"] = (
            out["total_isolados"] / total_pacientes_com_amostra * 1000
        ).round(3)
    else:
        out["Taxa de Incidência (por 1000 pacientes)"] = 0

    out["Taxa de Resistência (%)"] = (
        out["resistentes"] / out["total_isolados"].replace(0, pd.NA) * 100
    ).round(2)

    cols = [
        "microorganismo",
        "Taxa de Incidência (por 1000 pacientes)",
        "Taxa de Resistência (%)",
    ]
    return out[cols]

In [8]:
# Célula 8: Função principal para gerar Excel (AJUSTE NECESSÁRIO)
def gerar_planilhas_cciras(
    csv_in: str, pos_counts: str, neg_counts: str, xlsx_out: str
):
    df = read_csv_robusto(csv_in)
    # A chamada a carregar_contagens agora retorna inteiros
    total_pos, total_neg = carregar_contagens(Path(pos_counts), Path(neg_counts))

    p1 = planilha_isolados(df)
    p2 = planilha_sensibilidade(df)
    p3 = planilha_tendencia(df)
    # A chamada a planilha_indicadores foi atualizada com os novos parâmetros
    p4 = planilha_indicadores(df, total_pos, total_neg)

    with pd.ExcelWriter(xlsx_out, engine="openpyxl") as wr:
        p1.to_excel(wr, sheet_name="Isolados Detalhados", index=False)
        p2.to_excel(wr, sheet_name="Sensibilidade", index=False)
        p3.to_excel(wr, sheet_name="Tendência Temporal", index=False)
        p4.to_excel(wr, sheet_name="Indicadores", index=False)
    print(f"✔ Gerado: {xlsx_out}")

In [9]:
# Célula 9: Processar todos os arquivos do ano (opcional)
def processar_todos_arquivos_ano(ano_2d: str, pos_counts: str, neg_counts: str):
    import re, os

    r = re.compile(rf"vigiram-[a-z]{{3}}{ano_2d}\.csv$", re.IGNORECASE)
    for nome_arq in os.listdir(DATA_DIR):
        if r.match(nome_arq):
            csv_in = DATA_DIR / nome_arq
            base = os.path.splitext(nome_arq)[0]
            xlsx_out = OUT_DIR / f"CCIRAS_{base}.xlsx"
            gerar_planilhas_cciras(csv_in, pos_counts, neg_counts, xlsx_out)
    print("✔ Finalizado para todos os arquivos do ano 20" + ano_2d)

In [10]:
# Célula 10: Execução do Processo
processar_todos_arquivos_ano(
    "25",
    DATA_DIR / "Contagem pacientes amostra positiva.csv",
    DATA_DIR / "Contagem pacientes amostras negativas.csv",
)

✔ Gerado: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\CCIRAS_vigiram-abr25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\CCIRAS_vigiram-fev25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\CCIRAS_vigiram-jan25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\CCIRAS_vigiram-jun25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\CCIRAS_vigiram-mai25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\CCIRAS_vigiram-mar25.xlsx
✔ Finalizado para todos os arquivos do ano 2025
